In [3]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D,
    Flatten, Dense, Dropout, SpatialDropout2D, BatchNormalization, Activation
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# EEGNet definition with modifications
def EEGNet(nb_classes, Chans=22, Samples=1000, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16):
    input1 = Input(shape=(Chans, Samples, 1))

    block1 = Conv2D(F1, (1, kernLength), padding='same', use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False,
                             depth_multiplier=D,
                             depthwise_regularizer=l2(0.001),
                             padding='valid')(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('swish')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = SpatialDropout2D(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('swish')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = SpatialDropout2D(dropoutRate)(block2)

    flatten = Flatten()(block2)
    dense = Dense(64, activation='swish')(flatten)
    output = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(0.001))(dense)

    model = Model(inputs=input1, outputs=output)
    return model

# Load and preprocess data
df = pd.read_csv(r"E:\sem8\Final\NEW_TRY\BCICIV_2a_all_patients_normalized1.csv")
eeg_channels = [col for col in df.columns if col.startswith('EEG-')]
grouped = df.groupby(['patient', 'epoch'])

X, y = [], []
label_map = {'left': 0, 'right': 1, 'foot': 2, 'tongue': 3}

for (patient, epoch), group in grouped:
    group_sorted = group.sort_values(by='time')
    data = group_sorted[eeg_channels].values
    if data.shape[0] < 1000:
        data = np.pad(data, ((0, 1000 - data.shape[0]), (0, 0)), mode='constant')
    else:
        data = data[:1000]
    X.append(data)
    label = group_sorted['label'].iloc[0]
    y.append(label_map[label])

X = np.array(X)
y = to_categorical(y, num_classes=4)

# Normalize per trial
X -= np.mean(X, axis=(1, 2), keepdims=True)
X /= np.std(X, axis=(1, 2), keepdims=True) + 1e-6

# Reshape for EEGNet
X = np.transpose(X, (0, 2, 1))
X = X[..., np.newaxis]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y.argmax(1), random_state=42)

# Compile model
model = EEGNet(nb_classes=4, Chans=22, Samples=1000)
model.compile(loss='categorical_crossentropy', optimizer=Adam(5e-4), metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Train
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)

# Evaluate
y_pred = model.predict(X_test).argmax(1)
print(classification_report(y_test.argmax(1), y_pred))
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc * 100:.2f}%")


Epoch 1/100


KeyboardInterrupt: 

In [4]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, DepthwiseConv2D, SeparableConv2D, AveragePooling2D,
    Flatten, Dense, Dropout, SpatialDropout2D, BatchNormalization, Activation
)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# EEGNet definition with modifications
def EEGNet(nb_classes, Chans=22, Samples=1000, dropoutRate=0.5, kernLength=64, F1=8, D=2, F2=16):
    input1 = Input(shape=(Chans, Samples, 1))

    block1 = Conv2D(F1, (1, kernLength), padding='same', use_bias=False)(input1)
    block1 = BatchNormalization()(block1)
    block1 = DepthwiseConv2D((Chans, 1), use_bias=False,
                             depth_multiplier=D,
                             depthwise_regularizer=l2(0.001),
                             padding='valid')(block1)
    block1 = BatchNormalization()(block1)
    block1 = Activation('swish')(block1)
    block1 = AveragePooling2D((1, 4))(block1)
    block1 = SpatialDropout2D(dropoutRate)(block1)

    block2 = SeparableConv2D(F2, (1, 16), use_bias=False, padding='same')(block1)
    block2 = BatchNormalization()(block2)
    block2 = Activation('swish')(block2)
    block2 = AveragePooling2D((1, 8))(block2)
    block2 = SpatialDropout2D(dropoutRate)(block2)

    flatten = Flatten()(block2)
    dense = Dense(64, activation='swish')(flatten)
    output = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(0.001))(dense)

    model = Model(inputs=input1, outputs=output)
    return model

# Load and preprocess data
df = pd.read_csv(r"E:\sem8\Final\NEW_TRY\BCICIV_2a_all_patients_normalized1.csv")
eeg_channels = [col for col in df.columns if col.startswith('EEG-')]
grouped = df.groupby(['patient', 'epoch'])

X, y = [], []
label_map = {'left': 0, 'right': 1, 'foot': 2, 'tongue': 3}

window_size = 1000
stride = 250

for (patient, epoch), group in grouped:
    group_sorted = group.sort_values(by='time')
    data = group_sorted[eeg_channels].values
    label = label_map[group_sorted['label'].iloc[0]]

    if data.shape[0] >= window_size:
        for start in range(0, data.shape[0] - window_size + 1, stride):
            end = start + window_size
            segment = data[start:end]
            X.append(segment)
            y.append(label)
    else:
        # Optional fallback: pad short trials
        padded = np.pad(data, ((0, window_size - data.shape[0]), (0, 0)), mode='constant')
        X.append(padded)
        y.append(label)


X = np.array(X)
if X.ndim == 3:
    X -= np.mean(X, axis=(1, 2), keepdims=True)
    X /= np.std(X, axis=(1, 2), keepdims=True) + 1e-6
else:
    raise ValueError(f"Unexpected X shape after conversion: {X.shape}")

y = to_categorical(y, num_classes=4)

# Normalize per trial
X -= np.mean(X, axis=(1, 2), keepdims=True)
X /= np.std(X, axis=(1, 2), keepdims=True) + 1e-6

# Reshape for EEGNet
X = np.transpose(X, (0, 2, 1))
X = X[..., np.newaxis]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y.argmax(1), random_state=42)

# Compile model
model = EEGNet(nb_classes=4, Chans=22, Samples=1000)
model.compile(loss='categorical_crossentropy', optimizer=Adam(5e-4), metrics=['accuracy'])

# Callbacks
early_stop = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# Train
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop, lr_scheduler],
    verbose=1
)

# Evaluate
y_pred = model.predict(X_test).argmax(1)
print(classification_report(y_test.argmax(1), y_pred))
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc * 100:.2f}%")


Epoch 1/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 13s 158ms/step - accuracy: 0.2708 - loss: 1.5150 - val_accuracy: 0.2602 - val_loss: 1.3972 - learning_rate: 5.0000e-04
Epoch 2/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 123ms/step - accuracy: 0.2495 - loss: 1.4348 - val_accuracy: 0.2602 - val_loss: 1.3976 - learning_rate: 5.0000e-04
Epoch 3/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 129ms/step - accuracy: 0.2628 - loss: 1.4251 - val_accuracy: 0.2602 - val_loss: 1.3966 - learning_rate: 5.0000e-04
Epoch 4/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 122ms/step - accuracy: 0.2952 - loss: 1.4127 - val_accuracy: 0.2602 - val_loss: 1.3978 - learning_rate: 5.0000e-04
Epoch 5/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 121ms/step - accuracy: 0.2656 - loss: 1.4118 - val_accuracy: 0.2602 - val_loss: 1.3956 - learning_rate: 5.0000e-04
Epoch 6/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 122ms/step - accuracy: 0.2854 - loss: 1.4008 - val_accuracy: 0.2577 - val_loss: 1.3923 - learning_rate: 5.0000e-04
Epoch 7/100
49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 123ms/step - ac